<a href="https://colab.research.google.com/github/Hafsa-Ugase/Hate-Speech-detection-/blob/main/Grid_search_for_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---


# <font color='#2E86C1'>**`Importing libraries`** </font>



---



In [ ]:
!pip install tweet-preprocessor
# importing libraries
import pandas as pd # imports the dataset csv.
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
import nltk
import string
import re 
import preprocessor as pr
import seaborn as sns


nltk.download('stopwords')
from nltk.corpus import stopwords
# one way to show the data in a string values
#",".join(stopwords.words('english'))
# another way to print out the data values
stop_wordsall= (stopwords.words('english'))
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
ps = nltk.PorterStemmer()

# importing models

from sklearn.model_selection import train_test_split,cross_validate , GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors  import KNeighborsClassifier




# importing 
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer ,TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from imblearn.over_sampling import SMOTE


from sklearn import metrics 
from sklearn import ensemble
from sklearn import linear_model


from sklearn import model_selection



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.




---


# <font color='#2E86C1'>**`Data Loading`** </font>




---



In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv")



---

# <font color='#2E86C1'>**`Data preprocessing`** </font>

---

In [ ]:
from typing_extensions import Text
def remove_pattern(text,pattern):
    
    # re.findall() finds the pattern i.e @user and puts it in a list for further task
    r = re.findall(pattern,text)

    # remove https links
    text = re.sub(r"http\S+", "", text)  
    # remove website links
    text = re.sub(r"www.\S+", "", text) 
     # Remove @
    text = re.sub(r'(@[A-Za-z0-9-_]+)', '', text)  
     # Lowercase
    text = text.lower()
  
     #re.sub() removes @user from the sentences in the dataset
    for i in r:
       text=  re.sub("@[A-Za-z0-9_]+","", text)    
    return text

# remove above mentioned
df['Clean_Tweets'] = np.vectorize(remove_pattern)(df['tweet'], "@[\w]*")

 # remove punction
df['Clean_Tweets'] = df['Clean_Tweets'].str.replace("[^a-zA-Z]", " ") 
df['Clean_Tweets'] = df['Clean_Tweets'].apply(lambda x: re.sub(r'\brt\b', '',x).strip())

#  tokenzie 
tokenized_tweet = df['Clean_Tweets'].apply(lambda x: x.split())
tokenized_tweet=  tokenized_tweet.apply(lambda x: [word for word in x if word not in stop_wordsall]) # stop word 
tokenized_tweet = tokenized_tweet.apply(lambda x: [ps.stem(i) for i in x]) # stem

for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

df['Clean_Tweets'] = tokenized_tweet


<ipython-input-4-ca80125df1bf>:28: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Clean_Tweets'] = df['Clean_Tweets'].str.replace("[^a-zA-Z]", " ")


---

# <font color='#2E86C1'>**`Applying the test on this two column`** </font>

---

In [ ]:
df_x =df ['Clean_Tweets']
df_y =df['class'].astype(int)



*  How the dataset looks like and is ready for testing.



In [ ]:
df_x.head(40)

0     woman complain clean hous amp man alway take t...
1     boy dat cold tyga dwn bad cuffin dat hoe st place
2            dawg ever fuck bitch start cri confus shit
3                                      look like tranni
4        shit hear might true might faker bitch told ya
5         shit blow claim faith somebodi still fuck hoe
6                 sit hate anoth bitch got much shit go
7               caus tire big bitch come us skinni girl
8                  amp might get ya bitch back amp that
9                       hobbi includ fight mariam bitch
10    keek bitch curv everyon lol walk convers like smh
11                           murda gang bitch gang land
12                            hoe smoke loser yea go ig
13                                 bad bitch thing like
14                                            bitch get
15                                     bitch nigga miss
16                                     bitch plz whatev
17                                           bit

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_x,df_y, random_state=42, test_size=0.2)

In [ ]:
count_vectorizer = CountVectorizer()

count = CountVectorizer(stop_words=stop_wordsall)

X_train_count = count.fit_transform(X_train)
X_test_count = count.transform(X_test)


---


# <font color='#2E86C1'>**`Grid search Ada boost`** </font>

---


In [ ]:
Final_a = AdaBoostClassifier(random_state=42 )

In [ ]:
param_grid = [ 
    
    {'learning_rate' : [0.1,0.5,1],
     
     'algorithm' : ['SAMME', 'SAMME.R'],

     'n_estimators' :[100,50,200]

    

    }
]

In [ ]:
ada_grid= GridSearchCV( Final_a , param_grid, 
 
 # https://stackoverflow.com/questions/66620269/i-got-the-warning-userwarning-one-or-more-of-the-test-scores-are-non-finite-w
   scoring= 'f1_micro',
   verbose= 10,
   cv = 5,
  
)

In [ ]:
ada_grid.fit(X_train_count,y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5; 1/18] START algorithm=SAMME, learning_rate=0.1, n_estimators=100.......
[CV 1/5; 1/18] END algorithm=SAMME, learning_rate=0.1, n_estimators=100;, score=0.775 total time=  23.7s
[CV 2/5; 1/18] START algorithm=SAMME, learning_rate=0.1, n_estimators=100.......
[CV 2/5; 1/18] END algorithm=SAMME, learning_rate=0.1, n_estimators=100;, score=0.775 total time=  23.1s
[CV 3/5; 1/18] START algorithm=SAMME, learning_rate=0.1, n_estimators=100.......
[CV 3/5; 1/18] END algorithm=SAMME, learning_rate=0.1, n_estimators=100;, score=0.775 total time=  21.8s
[CV 4/5; 1/18] START algorithm=SAMME, learning_rate=0.1, n_estimators=100.......
[CV 4/5; 1/18] END algorithm=SAMME, learning_rate=0.1, n_estimators=100;, score=0.775 total time=  23.0s
[CV 5/5; 1/18] START algorithm=SAMME, learning_rate=0.1, n_estimators=100.......
[CV 5/5; 1/18] END algorithm=SAMME, learning_rate=0.1, n_estimators=100;, score=0.775 total time=  23.1s
[CV 1/5; 

GridSearchCV(cv=5, estimator=AdaBoostClassifier(random_state=42),
             param_grid=[{'algorithm': ['SAMME', 'SAMME.R'],
                          'learning_rate': [0.1, 0.5, 1],
                          'n_estimators': [100, 50, 200]}],
             scoring='f1_micro', verbose=10)

---


# <font color='#2E86C1'>**`Grid search Random Forest`** </font>

---

In [ ]:
Final_r = RandomForestClassifier(class_weight='balanced', random_state=42, n_jobs= 1 )

In [ ]:
param_grid = [ 
  
    {
      "n_estimators" : [100,1500,100],

     'max_depth' : [1,20],
     'criterion' : ['gini' , 'entropy'],

    } 
]


In [ ]:
random_grid = GridSearchCV( Final_r , param_grid, 
 
 # https://stackoverflow.com/questions/66620269/i-got-the-warning-userwarning-one-or-more-of-the-test-scores-are-non-finite-w
   scoring= 'f1_micro',
   verbose= 10,
   cv = 5,
  
)

In [ ]:
random_grid.fit(X_train_count,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5; 1/12] START criterion=gini, max_depth=1, n_estimators=100..............
[CV 1/5; 1/12] END criterion=gini, max_depth=1, n_estimators=100;, score=0.736 total time=   1.2s
[CV 2/5; 1/12] START criterion=gini, max_depth=1, n_estimators=100..............
[CV 2/5; 1/12] END criterion=gini, max_depth=1, n_estimators=100;, score=0.745 total time=   0.7s
[CV 3/5; 1/12] START criterion=gini, max_depth=1, n_estimators=100..............
[CV 3/5; 1/12] END criterion=gini, max_depth=1, n_estimators=100;, score=0.708 total time=   0.7s
[CV 4/5; 1/12] START criterion=gini, max_depth=1, n_estimators=100..............
[CV 4/5; 1/12] END criterion=gini, max_depth=1, n_estimators=100;, score=0.787 total time=   0.7s
[CV 5/5; 1/12] START criterion=gini, max_depth=1, n_estimators=100..............
[CV 5/5; 1/12] END criterion=gini, max_depth=1, n_estimators=100;, score=0.736 total time=   0.7s
[CV 1/5; 2/12] START criterion=gini, max_dep

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(class_weight='balanced', n_jobs=1,
                                              random_state=42),
             param_grid=[{'criterion': ['gini', 'entropy'],
                          'max_depth': [1, 20],
                          'n_estimators': [100, 1500, 100]}],
             scoring='f1_micro', verbose=10)

---


# <font color='#2E86C1'>**`Grid search KNN`** </font>

---

In [ ]:
Final_k =KNeighborsClassifier()

In [ ]:
param_grid = [ 
    
    {'n_neighbors' : [3,4,5,7,9],
     
     'weights' : ['uniform', 'distance'],

     'algorithm' :['auto','kd_tree','brute','ball_tree']

    

    }
]

In [ ]:
knn_grid= GridSearchCV( Final_k , param_grid, 
 
 # https://stackoverflow.com/questions/66620269/i-got-the-warning-userwarning-one-or-more-of-the-test-scores-are-non-finite-w
   scoring= 'f1_micro',
   verbose= 10,
   cv = 5,
  
)

**it took 7 min**

In [ ]:
knn_grid.fit(X_train_count,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV 1/5; 1/40] START algorithm=auto, n_neighbors=3, weights=uniform.............
[CV 1/5; 1/40] END algorithm=auto, n_neighbors=3, weights=uniform;, score=0.842 total time=   2.7s
[CV 2/5; 1/40] START algorithm=auto, n_neighbors=3, weights=uniform.............
[CV 2/5; 1/40] END algorithm=auto, n_neighbors=3, weights=uniform;, score=0.852 total time=   2.0s
[CV 3/5; 1/40] START algorithm=auto, n_neighbors=3, weights=uniform.............
[CV 3/5; 1/40] END algorithm=auto, n_neighbors=3, weights=uniform;, score=0.846 total time=   1.9s
[CV 4/5; 1/40] START algorithm=auto, n_neighbors=3, weights=uniform.............
[CV 4/5; 1/40] END algorithm=auto, n_neighbors=3, weights=uniform;, score=0.859 total time=   1.9s
[CV 5/5; 1/40] START algorithm=auto, n_neighbors=3, weights=uniform.............
[CV 5/5; 1/40] END algorithm=auto, n_neighbors=3, weights=uniform;, score=0.853 total time=   1.9s
[CV 1/5; 2/40] START algorithm=auto, n

/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_base.py:557: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


[CV 1/5; 11/40] END algorithm=kd_tree, n_neighbors=3, weights=uniform;, score=0.842 total time=   2.3s
[CV 2/5; 11/40] START algorithm=kd_tree, n_neighbors=3, weights=uniform.........


/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_base.py:557: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


[CV 2/5; 11/40] END algorithm=kd_tree, n_neighbors=3, weights=uniform;, score=0.852 total time=   6.8s
[CV 3/5; 11/40] START algorithm=kd_tree, n_neighbors=3, weights=uniform.........


...

/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_base.py:557: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


[CV 4/5; 40/40] END algorithm=ball_tree, n_neighbors=9, weights=distance;, score=0.839 total time=   2.2s
[CV 5/5; 40/40] START algorithm=ball_tree, n_neighbors=9, weights=distance......


/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_base.py:557: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


[CV 5/5; 40/40] END algorithm=ball_tree, n_neighbors=9, weights=distance;, score=0.840 total time=   2.2s


GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid=[{'algorithm': ['auto', 'kd_tree', 'brute',
                                        'ball_tree'],
                          'n_neighbors': [3, 4, 5, 7, 9],
                          'weights': ['uniform', 'distance']}],
             scoring='f1_micro', verbose=10)

---


# <font color='#2E86C1'>**`Grid search Logestic regression`** </font>

---

In [ ]:
Final_l = LogisticRegression(class_weight='balanced', random_state=42 )

In [ ]:
param_grid = [ 
    
    {'penalty' : [ 'l2','l1','None','elasticnet'],
     
     'C': [100, 10, 1.0, 0.1, 0.01],

     'solver': ['liblinear', 'newton-cg', 'sag'],
    }
]

In [ ]:
lg_grid = GridSearchCV( Final_l , param_grid, 
 
 # https://stackoverflow.com/questions/66620269/i-got-the-warning-userwarning-one-or-more-of-the-test-scores-are-non-finite-w
   scoring= 'f1_micro',
   verbose= 10,
   cv = 5,
  
)

In [ ]:
lg_grid.fit(X_train_count,y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV 1/5; 1/60] START C=100, penalty=l2, solver=liblinear........................
[CV 1/5; 1/60] END C=100, penalty=l2, solver=liblinear;, score=0.854 total time=   4.9s
[CV 2/5; 1/60] START C=100, penalty=l2, solver=liblinear........................


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 2/5; 1/60] END C=100, penalty=l2, solver=liblinear;, score=0.872 total time=   4.5s
[CV 3/5; 1/60] START C=100, penalty=l2, solver=liblinear........................
[CV 3/5; 1/60] END C=100, penalty=l2, solver=liblinear;, score=0.858 total time=   3.6s
[CV 4/5; 1/60] START C=100, penalty=l2, solver=liblinear........................


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 4/5; 1/60] END C=100, penalty=l2, solver=liblinear;, score=0.864 total time=   5.6s
[CV 5/5; 1/60] START C=100, penalty=l2, solver=liblinear........................
[CV 5/5; 1/60] END C=100, penalty=l2, solver=liblinear;, score=0.868 total time=   1.6s
[CV 1/5; 2/60] START C=100, penalty=l2, solver=newton-cg........................
[CV 1/5; 2/60] END C=100, penalty=l2, solver=newton-cg;, score=0.836 total time=   4.2s
[CV 2/5; 2/60] START C=100, penalty=l2, solver=newton-cg........................
[CV 2/5; 2/60] END C=100, penalty=l2, solver=newton-cg;, score=0.843 total time=   6.4s
[CV 3/5; 2/60] START C=100, penalty=l2, solver=newton-cg........................
[CV 3/5; 2/60] END C=100, penalty=l2, solver=newton-cg;, score=0.844 total time=   4.2s
[CV 4/5; 2/60] START C=100, penalty=l2, solver=newton-cg........................
[CV 4/5; 2/60] END C=100, penalty=l2, solver=newton-cg;, score=0.848 total time=   4.0s
[CV 5/5; 2/60] START C=100, penalty=l2, solver=newton-cg...........

...

[CV 4/5; 27/60] END C=1.0, penalty=l2, solver=sag;, score=0.866 total time=   0.8s
[CV 5/5; 27/60] START C=1.0, penalty=l2, solver=sag.............................


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 5/5; 27/60] END C=1.0, penalty=l2, solver=sag;, score=0.875 total time=   0.8s
[CV 1/5; 28/60] START C=1.0, penalty=l1, solver=liblinear.......................
[CV 1/5; 28/60] END C=1.0, penalty=l1, solver=liblinear;, score=0.892 total time=   0.2s
[CV 2/5; 28/60] START C=1.0, penalty=l1, solver=liblinear.......................
[CV 2/5; 28/60] END C=1.0, penalty=l1, solver=liblinear;, score=0.902 total time=   0.3s
[CV 3/5; 28/60] START C=1.0, penalty=l1, solver=liblinear.......................
[CV 3/5; 28/60] END C=1.0, penalty=l1, solver=liblinear;, score=0.889 total time=   0.3s
[CV 4/5; 28/60] START C=1.0, penalty=l1, solver=liblinear.......................
[CV 4/5; 28/60] END C=1.0, penalty=l1, solver=liblinear;, score=0.893 total time=   0.3s
[CV 5/5; 28/60] START C=1.0, penalty=l1, solver=liblinear.......................
[CV 5/5; 28/60] END C=1.0, penalty=l1, solver=liblinear;, score=0.896 total time=   0.4s
[CV 1/5; 29/60] START C=1.0, penalty=l1, solver=newton-cg..........

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
200 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solver 

GridSearchCV(cv=5,
             estimator=LogisticRegression(class_weight='balanced',
                                          random_state=42),
             param_grid=[{'C': [100, 10, 1.0, 0.1, 0.01],
                          'penalty': ['l2', 'l1', 'None', 'elasticnet'],
                          'solver': ['liblinear', 'newton-cg', 'sag']}],
             scoring='f1_micro', verbose=10)